# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/victoray/PycharmProjects/DataModelingApacheCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [122]:
# TO-DO: Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [17]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifydb')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [117]:

session.execute('DROP TABLE IF EXISTS events_song')
session.execute('DROP TABLE IF EXISTS events_artist')
session.execute('DROP TABLE IF EXISTS events_user')

#TABLE 1: events_session
#PARTITION KEY: session_id
#CLUSTERING COLUMNS: item_in_session
#DESIGN: This table is designed to retrieve activities during different sessions
query = "CREATE TABLE IF NOT EXISTS events_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length double, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#TABLE 2: events_user
#PARTITION KEY: user_id
#CLUSTERING COLUMNS: session_id, item_in_session
#DESIGN: This table is designed to retrieve activities for users during different sessions in the app
query = "CREATE TABLE IF NOT EXISTS events_user"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#TABLE 3: events_song
#PARTITION KEY: song
#CLUSTERING COLUMNS: user_id
#DESIGN: This table is designed to retrieve songs played by different users in the app
query = "CREATE TABLE IF NOT EXISTS events_song"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [118]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## TO-DO: Assign the INSERT statements into the `query` variable
        #Insert Values into Table 1 
        query = "INSERT INTO events_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
        #Insert Values into Table 2
        query = "INSERT INTO events_user ( user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1], line[4]))
        
        
        #Insert Values into Table 3
        query = "INSERT INTO events_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


In [125]:
query = "SELECT * FROM events_session WHERE session_id = 338 AND item_in_session = 4"
result = session.execute(query)

for row in result:
    print(f'Artist: {row.artist}', f'Song: {row.song}', f'Length: {row.length}')

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: ConnectionException('Pool is shutdown')})

## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [120]:
query = "SELECT * FROM events_user WHERE user_id = 10 AND session_id = 182"
result = session.execute(query)

for row in result:
    print(f'Artist: {row.artist}', f'Song: {row.song}', f'name: {row.first_name + " " + row.last_name}')

Artist: Down To The Bone Song: Keep On Keepin' On name: Sylvie Cruz
Artist: Three Drives Song: Greece 2000 name: Sylvie Cruz
Artist: Sebastien Tellier Song: Kilometer name: Sylvie Cruz
Artist: Lonnie Gordon Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) name: Sylvie Cruz


## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [121]:
query = "SELECT * FROM events_song WHERE song = 'All Hands Against His Own'"
result = session.execute(query)

for row in result:
    print(f'name: {row.first_name + " " + row.last_name}')
             

name: Jacqueline Lynch
name: Tegan Levine
name: Sara Johnson


### Drop the tables before closing out the sessions

In [123]:
# Drop all tables
session.execute('DROP TABLE IF EXISTS events_session')
session.execute('DROP TABLE IF EXISTS events_song')
session.execute('DROP TABLE IF EXISTS events_user')

### Close the session and cluster connection¶

In [124]:
session.shutdown()
cluster.shutdown()